# Verizon chat analysis 

In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.cluster import KMeans

from sklearn.metrics import pairwise_distances_argmin_min


In [31]:
! pwd

/Users/akshay/Desktop/Fractal/NLP


In [3]:

df_chat=pd.read_csv('/Users/akshay/Desktop/Fractal/NLP/chat_table_2018_12_12to2018_12_13_v1.csv',)
df_chat.head()

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b..."
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ..."
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b..."


In [4]:
df_chat.isna().sum()

Unnamed: 0                     0
engagement_id                  0
scat_event_dt                  0
nps_score                   3000
chat_agent_name              299
cust_cb_sentiment_score     3000
agent_cb_sentiment_score    3000
customer_chat_text           642
agent_chat_text              642
dtype: int64

In [5]:
df_chat1 = df_chat.dropna(axis=0, subset=['customer_chat_text'])
df_chat1.isna().sum()

Unnamed: 0                     0
engagement_id                  0
scat_event_dt                  0
nps_score                   2358
chat_agent_name                0
cust_cb_sentiment_score     2358
agent_cb_sentiment_score    2358
customer_chat_text             0
agent_chat_text                0
dtype: int64

In [6]:
from nltk.tokenize import sent_tokenize

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import library
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import string
import textblob as tb

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akshay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def preprocess(text):
    result=sent_tokenize(text,language='english')
    result=tokenize(result,language='english')
    return result

#df_chat1['customer_clean']=df_chat1['customer_chat_text'].apply(preprocess)

In [8]:
# function for text preprocessing
def text_processing(row):
    review = re.sub('[^a-zA-Z]', ' ', row['customer_chat_text']) # match all strings that start with a letter, the second means match all strings that contain a non-letter
    review = review.lower() # lower case
    review = review.split() # split words
    wl = WordNetLemmatizer()  # bettwer way to remove suffix by keping meaning full words
    # remove most common english words in sentece 
    #review = [wl.lemmatize(word) for word in review if not word in set(stopwords.words('english'))] # aslo stops most common english words such as a,an, the in ,on etc
    review = [wl.lemmatize(word) for word in review]
    #review = ' '.join(review)
    return review

df_chat1['customer_clean1']=df_chat1.apply(text_processing,axis=1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [9]:
df_chat1.head()

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text,customer_clean1
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...,"[hello, i, would, like, to, add, activate, an,..."
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b...","[is, the, website, down, is, your, website, do..."
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ...","[why, doe, a, number, in, my, call, log, come,..."
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...,"[what, is, the, galaxy, note, xxxx, storage, c..."
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b...","[hello, jessica, apparently, there, is, an, is..."


In [10]:
# word2vec and fasttext
from gensim.models import Word2Vec
from gensim.models import FastText

model_w2v = Word2Vec(sentences=df_chat1.customer_clean1, size=1000, window=5, min_count=2, workers=4, sg=0)
#model_ft = FastText(sentences=df_chat1.customer_clean1, size=1000, window=5, min_count=2, workers=4,sg=0)


In [11]:
# create dictionary of words index pairs
lst_w2v = model_w2v.wv.index2word
dic_w2v = {ii: indx for indx, ii in enumerate(lst_w2v)}
len(dic_w2v)

4030

In [12]:
w2v_matrix=model_w2v.wv.syn0
w2v_matrix.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(4030, 1000)

# tfidf

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer

# function for text preprocessing
def text_processing(row):
    review = re.sub('[^a-zA-Z]', ' ', row['customer_chat_text']) # match all strings that start with a letter, the second means match all strings that contain a non-letter
    review = review.lower() # lower case
    review = review.split() # split words
    wl = WordNetLemmatizer()  # bettwer way to remove suffix by keping meaning full words
    # remove most common english words in sentece 
    #review = [wl.lemmatize(word) for word in review if not word in set(stopwords.words('english'))] # aslo stops most common english words such as a,an, the in ,on etc
    review = [wl.lemmatize(word) for word in review]
    review = ' '.join(review)
    return review


df_chat1['customer_clean2']=df_chat1.apply(text_processing,axis=1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer

#TF-IDF vectorization ( with bigram )
#tf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tf = TfidfVectorizer(ngram_range=(1, 1),vocabulary=dic_w2v)
tf_matrix = tf.fit_transform(df_chat1.customer_clean2)

In [15]:
# features name
lst_tfidf = tf.get_feature_names()
x_df = pd.DataFrame(tf_matrix.toarray(), columns=lst_tfidf)
x_df.columns

Index(['i', 'the', 'to', 'it', 'a', 'my', 'you', 'and', 'is', 'for',
       ...
       'qualifies', 'renews', 'postal', 'fsi', 'cache', 'bump', 'gendron',
       'lodovico', 'oldani', 'rodella'],
      dtype='object', length=4030)

In [16]:
## check both words and index are same in tfidf and word2vec output
ff1 = [0 if lst_w2v[ii]==x_df.columns[ii] else 1 for ii in range(len(lst_w2v))]
ff1.count(0)

4030

In [25]:
m_tf = np.asmatrix(tf_matrix.toarray())
m.shape

(2358, 4030)

In [26]:
n_w2v=np.asmatrix(w2v_matrix)
n.shape

(4030, 1000)

In [28]:
tf_w2v=m_tf*n_w2v

In [29]:
tf_w2v.shape

(2358, 1000)

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

def cluster(text_encoded):
    n_clusters = int(np.ceil(len(text_encoded)**0.5))
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(text_encoded)
    return kmeans

df_chat['Customer_clean_cluster']=df_chat['Customer_clean_encoder'].apply(cluster)


